# Data Wrangling


### 0. Data Source
The housing prices for 2018 are scraped from [reiwa.com](https://reiwa.com.au/the-wa-market/perth-suburbs-price-data/). The geological information of Perth suburbs is scraped from [Postcodes Australia](https://postcodes-australia.com) and Wikipedia.

The venue information is pulled from [Foursquare API](https://foursquare.com/).

In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import re
import numpy as np

### 1. Scrape housing prices

In [2]:
url = 'https://reiwa.com.au/the-wa-market/perth-suburbs-price-data/' # create the request url
results = requests.get(url).text # make the GET request
soup = BeautifulSoup(results, 'html.parser')

# find the pricing table
table = [x for x in soup.find(class_="table-responsive").get_text().split('\n') if x]
table

['SUBURBMedian House PriceGrowth Percentage(Last 12 Months) View FullProfile',
 'Alexander Heights',
 '$430,000',
 '3.60%',
 'Alfred Cove',
 '$805,000',
 '5.20%',
 'Alkimos',
 '$400,000',
 '-1.50%',
 'Anketell',
 '$0',
 '0.00%',
 'Applecross',
 '$1,600,000',
 '0.00%',
 'Ardross',
 '$1,050,000',
 '0.00%',
 'Armadale',
 '$240,000',
 '-5.90%',
 'Ascot',
 '$710,000',
 '7.60%',
 'Ashby',
 '$460,500',
 '0.10%',
 'Ashfield',
 '$430,000',
 '-9.50%',
 'Attadale',
 '$1,033,500',
 '-9.90%',
 'Atwell',
 '$507,000',
 '-2.60%',
 'Aubin Grove',
 '$532,000',
 '-3.30%',
 'Aveley',
 '$445,000',
 '-2.80%',
 'Bailup',
 '$0',
 '0.00%',
 'Balcatta',
 '$487,750',
 '-2.40%',
 'Baldivis',
 '$390,000',
 '-3.60%',
 'Balga',
 '$315,000',
 '-6.70%',
 'Ballajura',
 '$400,000',
 '-5.90%',
 'Banjup',
 '$0',
 '0.00%',
 'Banksia Grove',
 '$382,500',
 '-2.90%',
 'Baskerville',
 '$0',
 '0.00%',
 'Bassendean',
 '$475,000',
 '-5.90%',
 'Bateman',
 '$705,000',
 '-1.90%',
 'Bayswater',
 '$570,000',
 '0.00%',
 'Beaconsfield',

In [3]:
# group the information in table by suburbs
house_prices = [table[i:i+3] for i in range(1, len(table), 3)]

# convert the pricing list into a dataframe
perth_house_prices = pd.DataFrame(house_prices)
perth_house_prices.columns = ['Suburb', 'Medium House Price', 'Growth Percentage']
perth_house_prices.head()

,Suburb,Medium House Price,Growth Percentage
0,Alexander Heights,"$430,000",3.60%
1,Alfred Cove,"$805,000",5.20%
2,Alkimos,"$400,000",-1.50%
3,Anketell,$0,0.00%
4,Applecross,"$1,600,000",0.00%


In [4]:
perth_house_prices.shape

(351, 3)

There was information of 351 suburbs in total scraped from this website. 

In [5]:
# data formating
perth_house_prices.replace(['\$', ',', '%'], '', inplace=True, regex=True) # Remove special charactors
perth_house_prices = perth_house_prices.astype({'Medium House Price': 'int', 'Growth Percentage': 'float'}) # Convert data types
perth_house_prices.head()

,Suburb,Medium House Price,Growth Percentage
0,Alexander Heights,430000,3.6
1,Alfred Cove,805000,5.2
2,Alkimos,400000,-1.5
3,Anketell,0,0.0
4,Applecross,1600000,0.0


In [6]:
sum(perth_house_prices['Medium House Price'] == 0) # the number of rows with 0 as their medium house prices

49

There are 49 suburbs with 0 as the price, which is certainly due to the missing of data. It's better to drop them since they are not useful.

In [7]:
# drop rows with 0 as their medium house prices
perth_house_prices = perth_house_prices[perth_house_prices['Medium House Price'] > 0].reset_index(drop=True)
perth_house_prices.head()

,Suburb,Medium House Price,Growth Percentage
0,Alexander Heights,430000,3.6
1,Alfred Cove,805000,5.2
2,Alkimos,400000,-1.5
3,Applecross,1600000,0.0
4,Ardross,1050000,0.0


### 2. Scrape geological information

I found some useful geological information on [Postcodes Australia](https://postcodes-australia.com). Since it's require to put the federal division of each suburb in the url, and that the federal division it requires does not match the official current one of the suburb, I decided to try every federal division for every suburb.

In [8]:
# federal divitions in Perth
federal_div = [x.lower() for x in ['Brand', 'Burt', 'Canning', 'Cowan', 'Curtin', 'Durack', 'Forrest', 'Fremantle', 'Hasluck', 'Moore', 'Pearce', 'Perth', 'Stirling', 'Swan', 'Tangney', 'Kalgoorlie', 'Melville']]
federal_div

['brand',
 'burt',
 'canning',
 'cowan',
 'curtin',
 'durack',
 'forrest',
 'fremantle',
 'hasluck',
 'moore',
 'pearce',
 'perth',
 'stirling',
 'swan',
 'tangney',
 'kalgoorlie',
 'melville']

In [9]:
def GetGeoInfo(suburb, fed_div):
    url = 'https://postcodes-australia.com/areas/wa/{}/{}'.format(fed_div, suburb) # create request url
    page = requests.get(url).text # make the GET request
    soup = BeautifulSoup(page, 'html.parser')
    
    details = soup.get_text().split('\n') # get a list Containing all useful information
    lat_lng = [re.findall(r'-*[0-9.]+', x) for x in details if re.match('Latitude', x)] # find latitude and longitude
    description = [x for x in details if re.match('Nearest', x)] # find the nearest urban center, airport, train station
    
    # if latitude and longitude were found, pass them into lat_lng
    # otherwise raise error to try the next federal division / suburb
    if lat_lng:
        lat_lng = lat_lng[0]
    else:
        raise NameError

    n_urbanc = description[0].split('(')[1].split('km away)')[0] # get the distance to the nearest urban center
    
    n_airport = description[1].split('Nearest Airport: ')[1].split('km away)')[0] # find the nearest airport
    n_airport = re.search(r'[0-9.]+', n_airport)
    
    n_train_st = description[1].split('Nearest Train Station: ')[1].split('km away)')[0] # find the nearest train station
    n_train_st = re.search(r'[0-9.]+', n_train_st)
    
    # get the distance to the nearest aiport / train station, if any can be found
    if n_airport:
        n_airport = n_airport.group() 
    if n_train_st:
        n_train_st = n_train_st.group()
    
    # return latitude, longitude, distances to the nearest urban center, airport and train station
    return lat_lng, n_urbanc, n_airport, n_train_st

In [17]:
for i in range(len(perth_house_prices)):
    suburb = perth_house_prices['Suburb'][i].lower().replace(' ', '+')
    
    # initiate values for each suburb
    j = 0
    lat_lng, n_urbanc, n_airport, n_train_st = [np.nan, np.nan], np.nan, np.nan, np.nan
    
    # try different federal divisions until all have been tried or information has been found
    while True:
        try:
            lat_lng, n_urbanc, n_airport, n_train_st = GetGeoInfo(suburb, federal_div[j])
        except NameError:
            j += 1
            if j < len(federal_div):
                continue
            else:
                break
        break
    
    # pass latitude, longitude, distances to the nearest urban center, airport and train station to the existing dataframe
    perth_house_prices.loc[i, 'Latitude'] = lat_lng[0]
    perth_house_prices.loc[i, 'Longitude'] = lat_lng[1]
    perth_house_prices.loc[i, 'Nearest Urban Center'] = n_urbanc
    perth_house_prices.loc[i, 'Nearest Airport'] = n_airport
    perth_house_prices.loc[i, 'Nearest Train Station'] = n_train_st

perth_house_prices.head(10)

,Suburb,Medium House Price,Growth Percentage,Latitude,Longitude,Nearest Urban Center,Nearest Airport,Nearest Train Station
0,Alexander Heights,430000,3.6,-31.828,115.867,14,16,7
1,Alfred Cove,805000,5.2,-32.034,115.809,10,18,3.7
2,Alkimos,400000,-1.5,-31.629,115.688,40,43,8.3
3,Applecross,1600000,0.0,-32.016,115.836,7,15,2
4,Ardross,1050000,0.0,-32.025,115.841,7.9,15,2.2
5,Armadale,240000,-5.9,-32.152,116.008,26,24,0.5
6,Ascot,710000,7.6,-31.941,115.923,6.3,4.2,2.7
7,Ashby,460500,0.1,-31.734,115.792,25,28,2.6
8,Ashfield,430000,-9.5,-31.915,115.938,8.7,3.9,0.3
9,Attadale,1033500,-9.9,-32.026,115.801,9.5,18,4.6


In [19]:
perth_house_prices[perth_house_prices.isnull().any(axis=1)] # get all the rows with missing data

,Suburb,Medium House Price,Growth Percentage,Latitude,Longitude,Nearest Urban Center,Nearest Airport,Nearest Train Station
12,Aveley,445000,-2.8,NaN,NaN,NaN,NaN,NaN
30,Bennett Springs,430000,-5.1,NaN,NaN,NaN,NaN,NaN
38,Brabham,420000,5.0,NaN,NaN,NaN,NaN,NaN
45,Bushmead,670000,0.0,NaN,NaN,NaN,NaN,NaN
49,Camillo,239000,-11.5,NaN,NaN,NaN,NaN,NaN
66,Cockburn Central,1465000,120.8,NaN,NaN,NaN,NaN,NaN
82,Dayton,420000,-9.8,NaN,NaN,NaN,NaN,NaN
113,Harrisdale,502000,-3.5,NaN,NaN,NaN,NaN,NaN
114,Haynes,444000,-14.6,NaN,NaN,NaN,NaN,NaN
122,Hilbert,380000,0.0,NaN,NaN,NaN,NaN,NaN


In [20]:
len(perth_house_prices[perth_house_prices.isnull().any(axis=1)]) # get the number of rows with missing data

13

There are 13 suburbs in total not found on [Postcodes Australia](https://postcodes-australia.com). Their latitude and longitude information can be easily found on Wikipedia, while the other information is much harder to find. It's easier to do without them when analyzing the relationship between housing prices and distances to the nearest urban center, nearest airport and nearest train station.

In [21]:
def GetLatLngWiki(suburb):
    url_wiki = 'https://en.wikipedia.org/wiki/{},_Western_Australia'.format(suburb) # create request url for Wikipedia
    wikipage = requests.get(url_wiki).text # request the webpage content
    soup = BeautifulSoup(wikipage, 'html.parser')
    
    # find latitude and longitude information in the content
    lat_lng = re.findall(r'[0-9.]+', soup.find('span', class_="geo-dec").get_text())
    lat_lng[0] = '-' + lat_lng[0] # put '-' befor the latitude information since Perth is in the southern hemisphere
    # return latitude and longitude
    return lat_lng

In [22]:
for i in perth_house_prices[perth_house_prices.isnull().any(axis=1)].index:
    # get suburb name for the row with missing data
    suburb = perth_house_prices['Suburb'][i]
    
    # try scraping the latitude and longitude information, pass if failed
    try:
        perth_house_prices.loc[i, ['Latitude', 'Longitude']] = GetLatLngWiki(suburb)
    except:
        pass

perth_house_prices[perth_house_prices.isnull().any(axis=1)]

,Suburb,Medium House Price,Growth Percentage,Latitude,Longitude,Nearest Urban Center,Nearest Airport,Nearest Train Station
12,Aveley,445000,-2.8,-31.781,115.988,NaN,NaN,NaN
30,Bennett Springs,430000,-5.1,-31.859,115.945,NaN,NaN,NaN
38,Brabham,420000,5.0,-31.8270,115.9746,NaN,NaN,NaN
45,Bushmead,670000,0.0,NaN,NaN,NaN,NaN,NaN
49,Camillo,239000,-11.5,-32.115,116.002,NaN,NaN,NaN
66,Cockburn Central,1465000,120.8,-32.121,115.848,NaN,NaN,NaN
82,Dayton,420000,-9.8,-31.8518,115.9755,NaN,NaN,NaN
113,Harrisdale,502000,-3.5,-32.112,115.932,NaN,NaN,NaN
114,Haynes,444000,-14.6,-32.154,115.974,NaN,NaN,NaN
122,Hilbert,380000,0.0,-32.172,115.980,NaN,NaN,NaN


As we can see, there is no information about Bushmead (nor can I found any on other websites). It's better to just drop it.

In [23]:
perth_house_prices.drop(45, axis=0, inplace=True) # drop the row of "Bushmead"
perth_house_prices.reset_index(drop=True, inplace=True)
perth_house_prices[42:46]

,Suburb,Medium House Price,Growth Percentage,Latitude,Longitude,Nearest Urban Center,Nearest Airport,Nearest Train Station
42,Bullsbrook,326000,-6.9,-31.671,116.031,36,31,5.9
43,Burns Beach,800000,-3.0,-31.721,115.721,29,34,2.8
44,Burswood,805000,28.2,-31.960,115.903,4.2,6.4,0.3
45,Butler,381000,-4.7,-31.649,115.710,37,40,5.3


In [24]:
# convert the data into right type for further analysis
for col in ['Latitude', 'Longitude', 'Nearest Urban Center', 'Nearest Airport', 'Nearest Train Station']:
    perth_house_prices[col] = perth_house_prices[col].astype('float')

perth_house_prices.head()

,Suburb,Medium House Price,Growth Percentage,Latitude,Longitude,Nearest Urban Center,Nearest Airport,Nearest Train Station
0,Alexander Heights,430000,3.6,-31.828,115.867,14.0,16.0,7.0
1,Alfred Cove,805000,5.2,-32.034,115.809,10.0,18.0,3.7
2,Alkimos,400000,-1.5,-31.629,115.688,40.0,43.0,8.3
3,Applecross,1600000,0.0,-32.016,115.836,7.0,15.0,2.0
4,Ardross,1050000,0.0,-32.025,115.841,7.9,15.0,2.2


In [25]:
# save this dataframe to .csv file for further analysis
perth_house_prices.to_csv('../Data/Perth_Suburbs.csv')

### 3. Pull venue data

In [26]:
CLIENT_ID = 'CK5DC4RRNYO1EI0E0K5PYKGE50N2BLMXBYI4GOGOS1QEAXSB' # Foursquare ID
CLIENT_SECRET = 'U3IAK4QGWTNE3YYOEYJT3AMCWLIAW0EED05I5LN1JLTJFOCL' # Foursquare Secret
VERSION = '20180604' # Foursquare Version

In [29]:
def getNearbyVenues(names, latitudes, longitudes, radius=2000, LIMIT=150):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):   
        # create the Foursquare API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Suburb', 
                  'Suburb Latitude', 
                  'Suburb Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [30]:
# get nearby venues for all the suburbs in perth_house_prices dataframe
perth_venues = getNearbyVenues(perth_house_prices['Suburb'], perth_house_prices['Latitude'], perth_house_prices['Longitude'])
perth_venues.head()

,Suburb,Suburb Latitude,Suburb Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Alexander Heights,-31.828,115.867,The Market Place Ballajura IGA,-31.826871,115.878213,Grocery Store
1,Alexander Heights,-31.828,115.867,Domino's Pizza,-31.830509,115.853342,Pizza Place
2,Alexander Heights,-31.828,115.867,Subway,-31.834327,115.879163,Sandwich Place
3,Alexander Heights,-31.828,115.867,Koondoola IGA X-press,-31.841556,115.859207,Grocery Store
4,Alexander Heights,-31.828,115.867,Alexander Heights Shopping Centre,-31.829769,115.853448,Shopping Mall


In [32]:
print('There are {} uniques categories.'.format(len(perth_venues['Venue Category'].unique())))

There are 281 uniques categories.


In [33]:
# create dummy variables of all the categorical variable for further analysis
perth_onehot = pd.get_dummies(perth_venues[['Venue Category']], prefix="", prefix_sep="")
perth_onehot['Suburb'] = perth_venues['Suburb']

# group all the dummy variables by suburbs, and calculate the ratio of venue category to all venue categories in a suburb
perth_grouped = perth_onehot.groupby('Suburb').mean()
perth_grouped.head()

,African Restaurant,Airport,Airport Gate,Airport Terminal,American Restaurant,Animal Shelter,Apres Ski Bar,Aquarium,Arcade,Art Gallery,...,Vineyard,Water Park,Waterfront,Whisky Bar,Wine Bar,Wine Shop,Winery,Wings Joint,Yoga Studio,Zoo
Suburb,,,,,,,,,,,,,,,,,,,,,
Alexander Heights,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Alfred Cove,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Alkimos,0.0,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Applecross,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Ardross,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.019231,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [34]:
# save this dataframe to .csv file for further analysis
perth_grouped.to_csv('../Data/Perth_Venues.csv')

Further analysis can be found in EDA.ipynb (Exploratory Data Analysis Jupyter Notebook) in the same folder.